In [1]:
#STEP 1: Install Required Libraries

# Install Hugging Face and Gradio
!pip install huggingface_hub
!pip install transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [2]:
#STEP 2: Import Python Libraries

# Import necessary libraries
from transformers import pipeline
import gradio as gr
import random


In [3]:
#STEP 3: Define Sample Course Data

# Sample electives stored as dictionary (no external dataset needed)
electives = {
    "data_science": {
        "title": "Data Science with Python",
        "description": "Covers data analysis, machine learning, and data visualization using Python.",
        "tags": ["data", "python", "machine learning", "visualization"]
    },
    "ai_fundamentals": {
        "title": "AI Fundamentals",
        "description": "Introduces AI techniques including search, planning, and neural networks.",
        "tags": ["ai", "intelligence", "search", "neural networks"]
    },
    "web_dev": {
        "title": "Web Development",
        "description": "Covers front-end and back-end web development using HTML, CSS, JS, and Flask.",
        "tags": ["web", "html", "javascript", "backend", "flask"]
    },
    "marketing_basics": {
        "title": "Marketing Basics",
        "description": "Fundamentals of digital and traditional marketing strategies.",
        "tags": ["marketing", "business", "branding"]
    }
}


In [4]:
#STEP 4: Load Hugging Face Language Model

# Load a text-generation model from Hugging Face
generator = pipeline("text-generation", model="distilgpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [5]:
#STEP 5: Define Course Recommendation Logic

# Function to select and explain a course recommendation
def recommend_elective(interests, career_goal, gpa):
    matches = []

    for course in electives.values():
        if any(tag in interests.lower() for tag in course["tags"]) or career_goal.lower() in course["description"].lower():
            matches.append(course)

    if not matches:
        return "❌ Sorry, I couldn’t find any course that matches your profile."

    chosen = random.choice(matches)

    prompt = f"Based on a student's interest in {interests} and goal to become a {career_goal}, explain why the course '{chosen['title']}' is a good choice."

    explanation = generator(prompt, max_length=100, do_sample=True, temperature=0.7)[0]['generated_text']

    return f"""
📘 **Recommended Course**: {chosen['title']}

📄 **Why this course?**
{explanation}
"""


In [6]:
#STEP 6: Define Full Advisor Agent Function

# Main function to be called by the interface
def smart_advisor(name, dept, gpa, interests, goal):
    recommendation = recommend_elective(interests, goal, gpa)
    return f"""
👋 Hello **{name}** from **{dept}** Department!

🎯 **GPA**: {gpa}
🧠 **Interests**: {interests}
🎓 **Career Goal**: {goal}

---

{recommendation}
"""


In [7]:
#STEP 7: Create Gradio Interface

# Build Gradio interface for user interaction
iface = gr.Interface(
    fn=smart_advisor,
    inputs=[
        gr.Textbox(label="Your Name"),
        gr.Dropdown(choices=["Engineering", "Management", "Data Science", "IT", "Biotech"], label="Department"),
        gr.Slider(minimum=0.0, maximum=4.0, step=0.1, label="Current GPA"),
        gr.Textbox(label="What topics are you interested in?"),
        gr.Textbox(label="What is your future career goal?")
    ],
    outputs="markdown",
    title="🎓 Smart Elective Advisor",
    description="Get AI-powered recommendations for your university elective courses! Built with Hugging Face Transformers and Gradio.",
    theme="default"
)


In [8]:
#STEP 8: Launch the App (with Public Link)

# Run the app and get a shareable link
iface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fce0a372c817d90c25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
